<a href="https://colab.research.google.com/github/ksdkamesh99/LowLightEnhancer/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
cd /content/drive/My Drive/LowLightEnhancement

/content/drive/My Drive/LowLightEnhancement


In [32]:
import tensorflow as tf
import numpy as np
from PIL import Image

In [33]:
img_high=np.load("image_high.npy")

In [34]:
img_low=np.load("image_low.npy")

In [35]:
img_high.shape

(1485, 150, 150, 3)

## Illumination Mask Network

In [36]:
top_inp=tf.keras.layers.Input(shape=(150,150,3))
top=tf.keras.layers.Conv2D(64,kernel_size=(3,3),input_shape=(150,150,3),activation='relu')(top_inp)
top=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(top)

In [37]:
bottom_inp=tf.keras.layers.Input(shape=(150,150,3))
bottom_resize=tf.keras.layers.Lambda( 
    lambda image: tf.compat.v1.image.resize_nearest_neighbor( 
        image, 
        (224, 224)
    )
)(bottom_inp)


In [38]:
bottom=tf.keras.layers.Conv2D(64,kernel_size=(3,3),input_shape=(224,224,3),activation='relu')(bottom_resize)
bottom=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(bottom)
bottom=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(bottom)
bottom=tf.keras.layers.Lambda( 
    lambda image: tf.compat.v1.image.resize_nearest_neighbor( 
        image, 
        (146,146)
    )
)(bottom)

In [39]:
bottom.get_shape()

TensorShape([None, 146, 146, 64])

In [40]:
top.get_shape()

TensorShape([None, 146, 146, 64])

In [41]:
merged=tf.keras.layers.concatenate([top,bottom],axis=-1)

In [42]:
merged=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(merged)
merged=tf.keras.layers.Conv2D(16,kernel_size=(3,3),activation='relu')(merged)
merged=tf.keras.layers.Conv2D(3,kernel_size=(3,3),activation='relu')(merged)

In [43]:
merged.get_shape()

TensorShape([None, 140, 140, 3])

In [44]:
model_illumination_mask=tf.keras.models.Model(inputs=[top_inp,bottom_inp],outputs=merged)

In [45]:
model_illumination_mask.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
lambda_34 (Lambda)              (None, 224, 224, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 222, 222, 64) 1792        lambda_34[0][0]                  
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 150, 150, 3) 0                                            
_______________________________________________________________________________________

# Illumination Map 

In [46]:
merged=tf.keras.layers.Lambda( 
    lambda image: tf.compat.v1.image.resize_nearest_neighbor( 
        image,
        (150,150)
    )
)(merged)

In [47]:
merged.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'lambda_36/ResizeNearestNeighbor:0' shape=(None, 150, 150, 3) dtype=float32>>

In [48]:
def ieb(input_feature):
  ieb1=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(input_feature)
  ieb1=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(ieb1)
  max_pool=tf.keras.layers.GlobalMaxPooling2D()(ieb1)
  avg_pool=tf.keras.layers.GlobalAveragePooling2D()(ieb1)
  dense1=tf.keras.layers.Dense(8,activation='relu',kernel_initializer='he_normal')
  dense2=tf.keras.layers.Dense(64,activation='sigmoid',kernel_initializer='he_normal')
  max_pool=dense1(max_pool)
  max_pool=dense2(max_pool)
  avg_pool=dense1(avg_pool)
  avg_pool=dense2(avg_pool)
  max_pool=tf.keras.layers.Lambda(
    lambda image: tf.expand_dims(tf.expand_dims(image,axis=1),axis=1))(max_pool)
  avg_pool=tf.keras.layers.Lambda(
    lambda image: tf.expand_dims(tf.expand_dims(image,axis=1),axis=1))(avg_pool)
  channel=tf.keras.layers.Add()([max_pool,avg_pool])
  ieb1=tf.keras.layers.Multiply()([ieb1,channel])
  max_pool_s=tf.keras.layers.Lambda(
    lambda x: tf.keras.backend.max(x,axis=3,keepdims=True))(ieb1)
  avg_pool_s=tf.keras.layers.Lambda(
    lambda x: tf.keras.backend.mean(x,axis=3,keepdims=True))(ieb1)
  concat_slayers=tf.keras.layers.Concatenate(axis=3)([avg_pool_s,max_pool_s])
  spacial=tf.keras.layers.Conv2D(1,7,activation='sigmoid')(concat_slayers)
  spacial=tf.keras.layers.Lambda( 
    lambda image: tf.compat.v1.image.resize_nearest_neighbor( 
        image, 
        (146,146)
    ))(spacial)
  ieb1=tf.keras.layers.Multiply()([ieb1,spacial])
  ieb1=tf.keras.layers.BatchNormalization()(ieb1)
  ieb1=tf.keras.layers.Activation('relu')(ieb1)
  ieb1=tf.keras.layers.Lambda(
    lambda image: tf.compat.v1.image.resize_nearest_neighbor(image,(150,150)))(ieb1)
  return ieb1


  



In [49]:
ieb_1=ieb(merged)

In [50]:
ieb_2=ieb(ieb_1)

In [51]:
ieb_3=ieb(ieb_2)

In [52]:
ieb_4=ieb(ieb_3)

In [53]:
ieb_5=ieb(ieb_4)

In [54]:
added_ieb=tf.keras.layers.Add()([ieb_1,ieb_2,ieb_3,ieb_4,ieb_5])

In [55]:
impnet=tf.keras.layers.Conv2D(64,(3,3),activation='relu')(added_ieb)

In [56]:
impnet=tf.keras.layers.Conv2D(16,(3,3),activation='relu')(impnet)

In [57]:
impnet=tf.keras.layers.Conv2D(3,(3,3),activation='relu')(impnet)

In [58]:
impnet=tf.keras.layers.Lambda(
    lambda image: tf.compat.v1.image.resize_nearest_neighbor(image,
                                                             (150,150))
)(impnet)

In [59]:
impnet

<tf.Tensor 'lambda_67/ResizeNearestNeighbor:0' shape=(None, 150, 150, 3) dtype=float32>

# Correction Network

In [60]:
def correction_network(input_feature):
  conv1=tf.keras.layers.Conv2D(64,kernel_size=(3,3),strides=(1,1),activation='relu')(input_feature)
  conv2=tf.keras.layers.Conv2D(64,kernel_size=(3,3),strides=(1,1),activation='relu')(conv1)
  conv3=tf.keras.layers.Conv2D(32,kernel_size=(3,3),strides=(1,1),activation='relu')(conv2)
  conv4=tf.keras.layers.Conv2D(32,kernel_size=(3,3),strides=(1,1),activation='relu')(conv3)
  conv5=tf.keras.layers.Conv2D(3,kernel_size=(3,3),strides=(1,1),activation='relu')(conv4)
  conv5=tf.keras.layers.Lambda(
      lambda image:tf.compat.v1.image.resize_nearest_neighbor(
          image,size=(150,150)
      )
  )(conv5)
  return conv5